In [1]:
import nipype
import os,glob,sys,shutil
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from IPython.display import Image

In [2]:
subjRootDir = "/data/HCP_Data/HCP_BedpostData/"
FULL_SUBJECT_LIST = [x for x in os.listdir(subjRootDir) if os.path.isdir( subjRootDir+x)]
print(len(FULL_SUBJECT_LIST),"Subjects are potentially available to be processed!")
SampleSubjList = ['329440']

899 Subjects are potentially available to be processed!


In [3]:
"""
Setup for Probtrackx2 Computational Pipeline
"""
subj_infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id']),  name="subj_infosource")
#infosource.iterables = ('subject_id', SampleSubjList)
subj_infosource.iterables = ('subject_id', FULL_SUBJECT_LIST[0:2])
### Above just converts the list of subjects into an iterable list I can connect to the next part of the pipeline

In [4]:
# """
# Setup for DataGrabber inputs needed for probtrackx2
# """
datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
        outfields=['nodif_brain_mask','xfm','invxfm','thsamples','phsamples','fsamples']),
        name='datasource')
# create a node to obtain the functional images
datasource.inputs.base_directory = "/data/HCP_Data/HCP_BedpostData/"
datasource.inputs.template ='*'
datasource.inputs.sort_filelist = True
datasource.inputs.field_template = dict(
    thsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    fsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    phsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    nodif_brain_mask='%s/T1w/Diffusion.bedpostX/%s.nii*', 
    xfm='%s/MNINonLinear/xfms/%s.nii*',
    invxfm='%s/MNINonLinear/xfms/%s.nii*'
    )

datasource.inputs.template_args = dict(
             thsamples = [['subject_id','th1samples']],
             phsamples =  [['subject_id','ph1samples']],
             fsamples =  [['subject_id','f1samples']],
             nodif_brain_mask = [['subject_id','nodif_brain_mask']],
             xfm = [['subject_id','acpc_dc2standard']],
             invxfm = [['subject_id', 'standard2acpc_dc']]           
        )

In [14]:
seedMask = "/data/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Bilat.nii.gz"
seedMaskList = glob.glob('/data/HCP_Data/EHECHT_ROIS/Hu*')
debug_output = False


if debug_output:
    pbx2 = fsl.ProbTrackX2()
    pbx2.inputs.thsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_th1samples.nii.gz'
    pbx2.inputs.phsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_ph1samples.nii.gz'
    pbx2.inputs.fsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_f1samples.nii.gz'
    pbx2.inputs.mask = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/nodif_brain_mask.nii.gz'
    pbx2.inputs.samples_base_name = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged'
else:
    pbx2 = pe.Node(interface=fsl.ProbTrackX2(), name='pbx2')

    
pbx2.inputs.c_thresh = 0.2   # -c 0.2   F cutoff
pbx2.inputs.n_steps = 2000   # -S 2000
pbx2.inputs.step_length = 0.5 # --steplength=0.5
pbx2.inputs.n_samples = 50 # -P 5000
pbx2.inputs.opd = True
pbx2.inputs.loop_check = True
# pbx2.inputs.fibst = True


pbx2.iterables = ("seed", seedMaskList)

## fibthresh
# pbx2.inputs.onewaycondition = True # --onewaycondition
pbx2.inputs.correct_path_distribution = True # -pd  i.e. distance correction
#pbx2.inputs.samples_base_name = 'merged'
#pbx2.inputs.lrtarget3= targetMask  ## This is a hack for now
pbx2.inputs.seed = seedMask
#pbx2.inputs.omatrix3 = True   # --omatrix3
#pbx2.inputs.target3 = targetMask # --target3=$WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_GM.nii.gz "; 


### #FIBTHRESH APPEARS MISSING FROM INTERFACE???

if debug_output:
    print( pbx2.cmdline)
    print("WE WILL BE PROCESSING THE FOLLOWING ROIS")
    print([os.path.basename(x) for x in seedMaskList])
    pbx2.inputs.os2t = True


In [ ]:
probtrackx2  --fibthresh=0.1 --randfib=0 

In [15]:
#runpbx.connect(subj_infosource,'subject_id',datasource,'subject_id')
runpbx2  = pe.Workflow(name="runpbx2")
runpbx2.base_dir = "/data/HCP_Data/NipypeScratch/"

samples_base_name_fxn = lambda x : x.replace('_th1samples.nii.gz','')


runpbx2.connect(subj_infosource,'subject_id',datasource,'subject_id')
# ### Connect the dti_datasource to the pbx2 command
runpbx2.connect( datasource,'thsamples',pbx2,'thsamples')
runpbx2.connect( datasource,'phsamples',pbx2,'phsamples')
runpbx2.connect( datasource,'fsamples',pbx2,'fsamples')
runpbx2.connect( datasource,'nodif_brain_mask',pbx2,'mask')
runpbx2.connect( datasource, ('thsamples', samples_base_name_fxn ), pbx2,'samples_base_name') ###  NOTE THIS IS A WEIRD TUPLE IS

runpbx2.run()

180711-20:37:15,592 workflow INFO:
	 Workflow runpbx2 settings: ['check', 'execution', 'logging', 'monitoring']
180711-20:37:15,624 workflow INFO:
	 Running serially.
180711-20:37:15,625 workflow INFO:
	 [Node] Setting-up "runpbx2.datasource" in "/data/HCP_Data/NipypeScratch/runpbx2/_subject_id_100307/datasource".
180711-20:37:15,640 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180711-20:37:15,660 workflow INFO:
	 [Node] Finished "runpbx2.datasource".
180711-20:37:15,663 workflow INFO:
	 [Node] Setting-up "runpbx2.pbx2" in "/data/HCP_Data/NipypeScratch/runpbx2/_subject_id_100307/_seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Right.nii.gz/pbx2".
180711-20:37:15,685 workflow INFO:
	 [Node] Running "pbx2" ("nipype.interfaces.fsl.dti.ProbTrackX2"), a CommandLine Interface with command:
probtrackx2 --cthr=0.200 --pd --forcedir --loopcheck -m /data/HCP_Data/HCP_BedpostData/100307/T1w/Diffusion.bedpostX/nodif_brain_mask.nii.gz --nsamples=50 --nste

180711-20:37:16,120 workflow WARNING:
	 [Node] Error on "runpbx2.pbx2" (/data/HCP_Data/NipypeScratch/runpbx2/_subject_id_100307/_seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Bilat.nii.gz/pbx2)
180711-20:37:16,125 workflow ERROR:
	 Node pbx2.aI.a3.b1 failed to run on host 145e230a3f42.
180711-20:37:16,128 workflow ERROR:
	 Saving crash info to /data/HCP_Data/Scripts/crash-20180711-203716-neuro-pbx2.aI.a3.b1-ef5cf9ce-351e-49d6-b151-8ac2bc90a438.pklz
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/nipype/pipeline/plugins/linear.py", line 44, in run
    node.run(updatehash=updatehash)
  File "/opt/conda/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 480, in run
    result = self._run_interface(execute=True)
  File "/opt/conda/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 564, in _run_interface
    return self._run_command(execute)
  File "/opt/conda/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.

180711-20:37:16,413 workflow INFO:
	 [Node] Setting-up "runpbx2.pbx2" in "/data/HCP_Data/NipypeScratch/runpbx2/_subject_id_100307/_seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Bilat.nii.gz/pbx2".
180711-20:37:16,430 workflow INFO:
	 [Node] Running "pbx2" ("nipype.interfaces.fsl.dti.ProbTrackX2"), a CommandLine Interface with command:
probtrackx2 --cthr=0.200 --pd --forcedir --loopcheck -m /data/HCP_Data/HCP_BedpostData/100307/T1w/Diffusion.bedpostX/nodif_brain_mask.nii.gz --nsamples=50 --nsteps=2000 --opd --dir=/data/HCP_Data/NipypeScratch/runpbx2/_subject_id_100307/_seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Bilat.nii.gz/pbx2 --samples=/data/HCP_Data/HCP_BedpostData/100307/T1w/Diffusion.bedpostX/merged --seed=/data/HCP_Data/EHECHT_ROIS/Human_BasalForebrain_Bilat.nii.gz --steplength=0.500
180711-20:37:16,464 interface INFO:
	 stderr 2018-07-11T20:37:16.464084:probtrackx2: error while loading shared libraries: libfslvtkio.so: cannot open shared object file: No 

180711-20:37:16,855 workflow ERROR:
	 Node pbx2.aI.a4.b0 failed to run on host 145e230a3f42.
180711-20:37:16,857 workflow ERROR:
	 Saving crash info to /data/HCP_Data/Scripts/crash-20180711-203716-neuro-pbx2.aI.a4.b0-33b5ab5d-9942-4975-9489-cedf79f16a63.pklz
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/nipype/pipeline/plugins/linear.py", line 44, in run
    node.run(updatehash=updatehash)
  File "/opt/conda/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 480, in run
    result = self._run_interface(execute=True)
  File "/opt/conda/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 564, in _run_interface
    return self._run_command(execute)
  File "/opt/conda/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 644, in _run_command
    result = self._interface.run(cwd=outdir)
  File "/opt/conda/lib/python3.6/site-packages/nipype/interfaces/base/core.py", line 521, in run
    runtime = self._run_interf

180711-20:37:17,141 workflow INFO:
	 [Node] Setting-up "runpbx2.pbx2" in "/data/HCP_Data/NipypeScratch/runpbx2/_subject_id_100206/_seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Left.nii.gz/pbx2".
180711-20:37:17,158 workflow INFO:
	 [Node] Running "pbx2" ("nipype.interfaces.fsl.dti.ProbTrackX2"), a CommandLine Interface with command:
probtrackx2 --cthr=0.200 --pd --forcedir --loopcheck -m /data/HCP_Data/HCP_BedpostData/100206/T1w/Diffusion.bedpostX/nodif_brain_mask.nii.gz --nsamples=50 --nsteps=2000 --opd --dir=/data/HCP_Data/NipypeScratch/runpbx2/_subject_id_100206/_seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Left.nii.gz/pbx2 --samples=/data/HCP_Data/HCP_BedpostData/100206/T1w/Diffusion.bedpostX/merged --seed=/data/HCP_Data/EHECHT_ROIS/Human_BasalForebrain_Left.nii.gz --steplength=0.500
180711-20:37:17,194 interface INFO:
	 stderr 2018-07-11T20:37:17.193910:probtrackx2: error while loading shared libraries: libfslvtkio.so: cannot open shared object file: No suc

180711-20:37:17,430 workflow ERROR:
	 could not run node: runpbx2.pbx2.aI.a5.b0
180711-20:37:17,431 workflow INFO:
	 crashfile: /data/HCP_Data/Scripts/crash-20180711-203716-neuro-pbx2.aI.a5.b0-9a2ba8d9-2362-41f2-8ae8-bd2f74ce5255.pklz
180711-20:37:17,432 workflow ERROR:
	 could not run node: runpbx2.pbx2.aI.a4.b0
180711-20:37:17,433 workflow INFO:
	 crashfile: /data/HCP_Data/Scripts/crash-20180711-203716-neuro-pbx2.aI.a4.b0-33b5ab5d-9942-4975-9489-cedf79f16a63.pklz
180711-20:37:17,434 workflow ERROR:
	 could not run node: runpbx2.pbx2.aI.a3.b0
180711-20:37:17,435 workflow INFO:
	 crashfile: /data/HCP_Data/Scripts/crash-20180711-203716-neuro-pbx2.aI.a3.b0-c734999e-4112-4ee0-ac8d-babe8965a44b.pklz
180711-20:37:17,436 workflow ERROR:
	 could not run node: runpbx2.pbx2.aI.a2.b0
180711-20:37:17,436 workflow INFO:
	 crashfile: /data/HCP_Data/Scripts/crash-20180711-203717-neuro-pbx2.aI.a2.b0-613e098b-05b1-4f1d-bff1-bf6f9ffb1ce7.pklz
180711-20:37:17,437 workflow ERROR:
	 could not run node: ru

RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:
[Mandatory]
fsamples: (a list of items which are an existing file name)
mask: (an existing file name)
        bet binary mask file in diffusion space
        flag: -m %s
phsamples: (a list of items which are an existing file name)
seed: (an existing file name or a list of items which are an existing
         file name or a list of items which are a list of from 3 to 3 items
         which are an integer (int or long))
        seed volume(s), or voxel(s)or freesurfer label file
        flag: --seed=%s
thsamples: (a list of items which are an existing file name)

[Optional]
args: (a string)
        Additional parameters to the command
        flag: %s
avoid_mp: (an existing file name)
        reject pathways passing through locations given by this mask
        flag: --avoid=%s
c_thresh: (a float)
        curvature threshold - default=0.2
        flag: --cthr=%.3f
colmask4: (an existing file name)
        Mask for columns of matrix4 (default=seed mask)
        flag: --colmask4=%s
correct_path_distribution: (a boolean)
        correct path distribution for the length of the pathways
        flag: --pd
dist_thresh: (a float)
        discards samples shorter than this threshold (in mm - default=0)
        flag: --distthresh=%.3f
distthresh1: (a float)
        Discards samples (in matrix1) shorter than this threshold (in mm -
        default=0)
        flag: --distthresh1=%.3f
distthresh3: (a float)
        Discards samples (in matrix3) shorter than this threshold (in mm -
        default=0)
        flag: --distthresh3=%.3f
environ: (a dictionary with keys which are a value of type 'str' and
         with values which are a value of type 'str', nipype default value:
         {})
        Environment variables
fibst: (an integer (int or long))
        force a starting fibre for tracking - default=1, i.e. first fibre
        orientation. Only works if randfib==0
        flag: --fibst=%d
fopd: (an existing file name)
        Other mask for binning tract distribution
        flag: --fopd=%s
force_dir: (a boolean, nipype default value: True)
        use the actual directory name given - i.e. do not add + to make a
        new directory
        flag: --forcedir
ignore_exception: (a boolean, nipype default value: False)
        Print an error message instead of throwing an exception in case the
        interface fails to run
inv_xfm: (a file name)
        transformation matrix taking DTI space to seed space (compulsory
        when using a warp_field for seeds_to_dti)
        flag: --invxfm=%s
loop_check: (a boolean)
        perform loop_checks on paths - slower, but allows lower curvature
        threshold
        flag: --loopcheck
lrtarget3: (an existing file name)
        Column-space mask used for Nxn connectivity matrix
        flag: --lrtarget3=%s
meshspace: ('caret' or 'freesurfer' or 'first' or 'vox')
        Mesh reference space - either "caret" (default) or "freesurfer" or
        "first" or "vox"
        flag: --meshspace=%s
mod_euler: (a boolean)
        use modified euler streamlining
        flag: --modeuler
n_samples: (an integer (int or long), nipype default value: 5000)
        number of samples - default=5000
        flag: --nsamples=%d
n_steps: (an integer (int or long))
        number of steps per sample - default=2000
        flag: --nsteps=%d
network: (a boolean)
        activate network mode - only keep paths going through at least one
        seed mask (required if multiple seed masks)
        flag: --network
omatrix1: (a boolean)
        Output matrix1 - SeedToSeed Connectivity
        flag: --omatrix1
omatrix2: (a boolean)
        Output matrix2 - SeedToLowResMask
        flag: --omatrix2
        requires: target2
omatrix3: (a boolean)
        Output matrix3 (NxN connectivity matrix)
        flag: --omatrix3
        requires: target3, lrtarget3
omatrix4: (a boolean)
        Output matrix4 - DtiMaskToSeed (special Oxford Sparse Format)
        flag: --omatrix4
onewaycondition: (a boolean)
        Apply waypoint conditions to each half tract separately
        flag: --onewaycondition
opd: (a boolean, nipype default value: True)
        outputs path distributions
        flag: --opd
os2t: (a boolean)
        Outputs seeds to targets
        flag: --os2t
out_dir: (an existing directory name)
        directory to put the final volumes in
        flag: --dir=%s
output_type: ('NIFTI_PAIR' or 'NIFTI_PAIR_GZ' or 'NIFTI_GZ' or
         'NIFTI')
        FSL output type
rand_fib: (0 or 1 or 2 or 3)
        options: 0 - default, 1 - to randomly sample initial fibres (with f
        > fibthresh), 2 - to sample in proportion fibres (with f>fibthresh)
        to f, 3 - to sample ALL populations at random (even if f<fibthresh)
        flag: --randfib=%d
random_seed: (a boolean)
        random seed
        flag: --rseed
s2tastext: (a boolean)
        output seed-to-target counts as a text file (useful when seeding
        from a mesh)
        flag: --s2tastext
sample_random_points: (a boolean)
        sample random points within seed voxels
        flag: --sampvox
samples_base_name: (a string, nipype default value: merged)
        the rootname/base_name for samples files
        flag: --samples=%s
seed_ref: (an existing file name)
        reference vol to define seed space in simple mode - diffusion space
        assumed if absent
        flag: --seedref=%s
simple: (a boolean)
        rack from a list of voxels (seed must be a ASCII list of
        coordinates)
        flag: --simple
step_length: (a float)
        step_length in mm - default=0.5
        flag: --steplength=%.3f
stop_mask: (an existing file name)
        stop tracking at locations given by this mask file
        flag: --stop=%s
target2: (an existing file name)
        Low resolution binary brain mask for storing connectivity
        distribution in matrix2 mode
        flag: --target2=%s
target3: (an existing file name)
        Mask used for NxN connectivity matrix (or Nxn if lrtarget3 is set)
        flag: --target3=%s
target4: (an existing file name)
        Brain mask in DTI space
        flag: --target4=%s
target_masks: (a list of items which are a file name)
        list of target masks - required for seeds_to_targets classification
        flag: --targetmasks=%s
terminal_output: ('stream' or 'allatonce' or 'file' or 'none')
        Control terminal output: `stream` - displays to terminal immediately
        (default), `allatonce` - waits till command is finished to display
        output, `file` - writes output to file, `none` - output is ignored
use_anisotropy: (a boolean)
        use anisotropy to constrain tracking
        flag: --usef
verbose: (0 or 1 or 2)
        Verbose level, [0-2].Level 2 is required to output particle files.
        flag: --verbose=%d
waycond: ('OR' or 'AND')
        Waypoint condition. Either "AND" (default) or "OR"
        flag: --waycond=%s
wayorder: (a boolean)
        Reject streamlines that do not hit waypoints in given order. Only
        valid if waycond=AND
        flag: --wayorder
waypoints: (an existing file name)
        waypoint mask or ascii list of waypoint masks - only keep paths
        going through ALL the masks
        flag: --waypoints=%s
xfm: (an existing file name)
        transformation matrix taking seed space to DTI space (either FLIRT
        matrix or FNIRT warp_field) - default is identity
        flag: --xfm=%s

In [ ]:
# runpbx2.connect( dti_datasource, ('thsamples',samples_base_name_fxn) ,pbx2,'samples_base_name')
# # Run tractography with warp to/from template space
# $statement .= " -s $WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/merged "; 
# $statement .= " -x $WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_WM.nii.gz ";  ## check downsample
# $statement .= " -l   --fibthresh=0.1 --randfib=0 ";  ## check # of samples
# $statement .= " --forcedir --opd ";
# $statement .= " --dir=$WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/WholeBrainMatrixConnectivity_TemplateSpace_fullres/ ";
# print "$statement \n";

In [ ]:
info = dict(
            thsamples = [['subject_id','th1samples']],
            phsamples =  [['subject_id','ph1samples']],
            fsamples =  [['subject_id','f1samples']],
            nodif_brain_mask = [['subject_id','nodif_brain_mask']],
            xfm = [['subject_id','acpc_dc2standard.nii.gz']],
            invxfm = [['subject_id', 'standard2acpc_dc.nii.gz' ]] 
            )

info_template  = {'nodif_brain_mask': '%s/data.bedpostX/%s.nii.gz',    
    'thsamples' : '%s/data.bedpostX/merged_%s.nii.gz',
    'phsamples' : '%s/data.bedpostX/merged_%s.nii.gz',
    'fsamples' : '%s/data.bedpostX/merged_%s.nii.gz',
    'xfm': "%s/xfms/%s",
    'invxfm' :"%s/xfms/%s"
}


dti_datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
                                               outfields=info.keys()),
                                                 name='dti_datasource')
dti_datasource.inputs.base_directory = subjRootDir 
dti_datasource.inputs.sort_filelist = True
#dti_datasource.inputs.template = '*'
dti_datasource.inputs.field_template = info_template

dti_datasource.inputs.template_args = info
## Just mapped each subject to the corresponding bvec,bvals, brain mask and preprocessed DWI data
### Create the Node for DTIFIT


In [ ]:
# >>> dg = DataGrabber(infields=['sid'], outfields=['func','struct','ref'])
# >>> dg.inputs.base_directory = '.'
# >>> dg.inputs.template = '%s/%s.nii'
# >>> dg.inputs.template_args['func'] = [['sid',['f3','f5']]]
# >>> dg.inputs.template_args['struct'] = [['sid',['struct']]]
# >>> dg.inputs.template_args['ref'] = [['sid','ref']]
# >>> dg.inputs.sid = 's1'

In [ ]:
### Sample Subject = /HCP_Data/HCP_BedpostData/106016/T1w/Diffusion.bedpostX


In [ ]:
SUBJECT_LIST = ['106016']

subject_id_infosource = pe.Node(util.IdentityInterface(fields=['subject_id']),
                                name='subject_id_infosource')
subject_id_infosource.iterables = ('subject_id', FULL_SUBJECT_LIST[0:40])

In [ ]:
datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
                                               outfields=['dwi', 'bvec', 'bval']),
                                                 name='datasource')
datasource.inputs.base_directory = subjRootDir 
datasource.inputs.sort_filelist = True

pbx2 = pe.Node(interface=fsl.ProbTrackX2(), name='pbx2')
pbx2.inputs.c_thresh = 0.2
pbx2.inputs.correct_path_distribution = True
pbx2.inputs.n_samples  = 5000
pbx2.inputs.loop_check = True
pbx2.inputs.step_length = 0.5
pbx2.inputs.rand_fib = 0

#probtrackx2 --omatrix2 


In [ ]:
# --pd   correct_path_distribution 
#-l
#-c 0.2    --->> c_thresh
#-S 2000   
#--steplength=0.5
#-P 5000    n_samples
#--fibthresh=0.1   
#--randfib=0  rand_fib
#-s /home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/Subj_026/data.bedpostX/merged 
#-m /home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/Subj_026/data/nodif_brain_mask.nii.gz
#-x /home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/ROIs/Human_BasalForebrain_Left.nii.gz 
#--target2=/home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/ROIs/MNI152_T1_1mm_brain_mask.nii.gz 
#--xfm=/home/ehecht/TEMPLETON/Templeton_Registration/xfms/Subj_026_Scan2_MNI_fnirt_struct_6dof-flirt_nodif.nii.gz
#--invxfm=/home/ehecht/TEMPLETON/Templeton_Registration/xfms/Subj_026_Scan2_nodif_6dof-flirt_struct_fnirt_MNI.nii.gz 
#--forcedir 
#--opd
#--dir=/home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/Subj_026/BasalForebrain_Left_TemplateSpace/

In [ ]:
#  pbx2.inputs.seed = 'seed_source.nii.gz'
# >>> pbx2.inputs.thsamples = 'merged_th1samples.nii.gz'
# >>> pbx2.inputs.fsamples = 'merged_f1samples.nii.gz'
# >>> pbx2.inputs.phsamples = 'merged_ph1samples.nii.gz'
# >>> pbx2.inputs.mask = 'nodif_brain_mask.nii.gz'
# >>> pbx2.inputs.out_dir = '.'
# >>> pbx2.inputs.n_samples = 3
# >>> pbx2.inputs.n_steps = 10
# >>> pbx2.cmdline

# datasource.inputs.template = '%s/T1w/Diffusion/%s'
# datasource.inputs.template_args = dict(dwi=[['subject_id', 'data.nii.gz']],
#                                        bvecs=[['subject_id', 'bvecs']],
#                                        bvals=[['subject_id', 'bvals']],
#                                        nodif_brain_mask=[['subject_id','nodif_brain_mask.nii.gz']])
## Just mapped each subject to the corresponding bvec,bvals, brain mask and preprocessed DWI data
### Create the Node for DTIFIT
# dtifit = pe.Node(interface=fsl.DTIFit(), name='dtifit')


# gen_fa = pe.Workflow(name="gen_fa")
# gen_fa.base_dir = '/data/HCP_Data/NipypeScratch/'
# gen_fa.connect(subject_id_infosource, 'subject_id', datasource, 'subject_id')

# gen_fa.connect(subject_id_infosource, 'subject_id', dtifit, 'base_name')
# gen_fa.connect(datasource, 'bvecs', dtifit, 'bvecs')
# gen_fa.connect(datasource, 'bvals', dtifit, 'bvals')
# gen_fa.connect(datasource, 'nodif_brain_mask', dtifit, 'mask')
# gen_fa.connect(datasource, 'dwi', dtifit, 'dwi')

In [ ]:
print(gen_fa.write_graph(graph2use='colored',simple_form=False))
Image('/data/HCP_Data/NipypeScratch/gen_fa/graph.png')
#Image('/data/HCP_Data/NipypeScratch/gen_fa/graph.png')

In [ ]:
gen_fa.run(plugin='MultiProc', plugin_args={'n_procs' : 20})


In [ ]:
#!ls /data/HCP_Data/HCP_BedpostData/106016/T1w/Diffusion/

In [ ]:
datasink = pe.Node(interface=nio.DataSink(), name="datasink")
datasink.inputs.base_directory = os.path.join(os.path.abspath(working_data_dir),
                                              'sri_results')
datasink.inputs.parameterization = False
gen_fa.connect(dtifit, 'FA', datasink, 'FA')
gen_fa.connect(dtifit, 'MD', datasink, 'MD')

In [ ]:
from traits.api import HasTraits, Directory, Bool
import traits.api as traits
from ....base import MetaWorkflow, load_config, register_workflow
from wip_diffusion import config as pconfig
from bips.workflows.base import BaseWorkflowConfig

"""
Part 1: MetaWorkflow
"""
mwf = MetaWorkflow()
mwf.help = """
Diffusion tracking workflow
===========================
"""
mwf.uuid = 'fda82554a43511e1b507001e4fb1404c'
mwf.tags = ['diffusion','dti','tracking']
mwf.script_dir = 'fmri'

"""
Part 2: Config
"""

class config(BaseWorkflowConfig):
    uuid = traits.Str(desc="UUID")
    desc = traits.Str(desc='Workflow description')
    # Directories
    sink_dir = Directory(mandatory=True, desc="Location where the BIP will store the results")
    
    # Subjects

    subjects= traits.List(traits.Str, mandatory=True, usedefault=True,
        desc="Subject id's. Note: These MUST match the subject id's in the \
                                Freesurfer directory. For simplicity, the subject id's should \
                                also match with the location of individual functional files.")
    # Preprocessing info
    preproc_config = traits.File(desc="preproc json file")

    #Advanced
    use_advanced_options = traits.Bool()
    advanced_script = traits.Code()
    save_script_only = traits.Bool(False)

def create_config():
    c = config()
    c.uuid = mwf.uuid
    c.desc = mwf.help
    return c

mwf.config_ui = create_config

"""
Part 3: View
"""

def create_view():
    from traitsui.api import View, Item, Group, CSVListEditor
    from traitsui.menu import OKButton, CancelButton
    view = View(Group(Item(name='uuid', style='readonly'),
        Item(name='desc', style='readonly'),
        label='Description', show_border=True),
        Group(Item(name='working_dir'),
            Item(name='sink_dir'),
            Item(name='crash_dir'),
            label='Directories', show_border=True),
        Group(Item(name='run_using_plugin'),
            Item(name='plugin', enabled_when="run_using_plugin"),
            Item(name='plugin_args', enabled_when="run_using_plugin"),
            Item(name='test_mode'),
            label='Execution Options', show_border=True),
        Group(Item(name='subjects', editor=CSVListEditor()),
            label='Subjects', show_border=True),
        Group(Item(name='preproc_config'),
            label='Track', show_border=True),
        Group(Item("use_advanced_options"),
            Item("advanced_script"),
            label="Advanced",show_border=True),
        buttons = [OKButton, CancelButton],
        resizable=True,
        width=1050)
    return view

mwf.config_view = create_view

"""
Part 4: Construct Workflow
"""

from ..scripts.diffusion_base import create_workflow

def get_dataflow(c):
    import nipype.pipeline.engine as pe
    import nipype.interfaces.io as nio
    datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
        outfields=['dwi','mask','bvecs','bvals',"reg","mean"]),
        name='datasource')
    # create a node to obtain the functional images
    datasource.inputs.base_directory = c.sink_dir
    datasource.inputs.template ='*'
    datasource.inputs.field_template = dict(dwi='%s/preproc/outputs/dwi/*',
        mask='%s/preproc/outputs/mask/*', bvecs='%s/preproc/outputs/bvecs/*',
        bvals='%s/preproc/outputs/bvals/*',reg='%s/preproc/outputs/bbreg/*.dat',
        mean='%s/preproc/outputs/mean/*.nii*')
    datasource.inputs.template_args = dict(dwi=[['subject_id']],
        mask=[['subject_id']],
        bvecs=[['subject_id']],
        bvals=[['subject_id']],
        mean=[["subject_id"]],
        reg=[["subject_id"]])
    return datasource

foo = pconfig()

def get_wf(c, prep_c=foo):
    import nipype.pipeline.engine as pe
    import nipype.interfaces.io as nio
    import nipype.interfaces.utility as niu
    workflow = create_workflow()
    datagrabber = get_dataflow(prep_c)
    inputspec = workflow.get_node('inputspec')
    workflow.connect(datagrabber,'mask',inputspec,'mask')
    workflow.connect(datagrabber,'dwi',inputspec,'dwi')
    workflow.connect(datagrabber,'bvecs',inputspec,'bvecs')
    workflow.connect(datagrabber,'bvals',inputspec,'bvals')
    workflow.connect(datagrabber,'reg',inputspec,'reg')
    workflow.connect(datagrabber,'mean',inputspec,'mean')
    workflow.inputs.inputspec.surf_dir=prep_c.surf_dir
    infosource = pe.Node(niu.IdentityInterface(fields=["subject_id"]),name='subject_names')
    workflow.connect(infosource,"subject_id",datagrabber, 'subject_id')
    workflow.connect(infosource,"subject_id",inputspec, 'subject_id')
    sinker = pe.Node(nio.DataSink(),name='sinker')
    outputspec=workflow.get_node('outputspec')
    workflow.connect(outputspec,'fdt_paths',sinker,'track.fdt_paths')
    workflow.connect(outputspec,'log',sinker,'track.log')
    workflow.connect(outputspec,'particle_files',sinker,'track.particle_files')
    workflow.connect(outputspec,'targets',sinker,'track.targets')
    workflow.connect(outputspec,'way_total',sinker,'track.way_total')
    sinker.inputs.base_directory=c.sink_dir
    workflow.connect(infosource,"subject_id",sinker,"container")

    if c.test_mode:
        infosource.iterables=("subject_id", [c.subjects[0]])
    else:
        infosource.iterables=("subject_id", c.subjects)
    workflow.base_dir = c.working_dir
    return workflow

mwf.workflow_function = get_wf

"""
Part 5: Main
"""

def main(config_file):
    c = load_config(config_file,config)

    prep_c = load_config(c.preproc_config, pconfig)

    workflow = get_wf(c,prep_c)

    if c.use_advanced_options:
        exec c.advanced_script

    if c.test_mode:
        workflow.write_graph()

    if c.run_using_plugin:
        workflow.run(plugin=c.plugin, plugin_args=c.plugin_args)
    else:
        workflow.run()

    return 1

mwf.workflow_main_function = main

"""
Part 6: Main
"""

register_workflow(mwf)

In [ ]:
from nipype.interfaces import fsl

subjDir = "/data/HCP_Data/HCP_BedpostData/100206/T1w/Diffusion.bedpostX/"

pbx2 = fsl.ProbTrackX2()
pbx2.inputs.seed = '/data/HCP_Data/EHECHT_ROIS/Human_BasalForebrain_Left.nii.gz'
pbx2.inputs.thsamples = subjDir + 'merged_th1samples.nii.gz'
pbx2.inputs.fsamples =  subjDir + 'merged_f1samples.nii.gz'
pbx2.inputs.phsamples =  subjDir + 'merged_ph1samples.nii.gz'
pbx2.inputs.mask =  subjDir + 'nodif_brain_mask.nii.gz'
pbx2.inputs.out_dir = '.'
pbx2.inputs.n_samples = 3
pbx2.inputs.n_steps = 10
pbx2.cmdline
pbx2.run()
#'probtrackx2 --forcedir -m nodif_brain_mask.nii.gz --nsamples=3 --nstep